In [1]:
import requests
from IPython.display import display, HTML
import json # Import json to pretty print API response for debugging if needed

# --- WeatherAPI Config ---
API_KEY = "876fd3d7689b415ea5310841231803"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def get_weather_data(city):
    """Fetches weather data from the WeatherAPI."""
    params = {
        "key": API_KEY,
        "q": city
    }
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status() # Raise an exception for HTTP errors
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}
    except json.JSONDecodeError:
        return {"error": "Could not decode JSON from API response."}

def display_weather(city_name):
    """Fetches and displays weather data with an attractive design."""
    data = get_weather_data(city_name)

    if "error" in data:
        html_output = f"""
        <div style="
            background-color: #ffe6e6;
            border-left: 6px solid #ff4d4d;
            padding: 10px;
            margin: 10px 0;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            color: #cc0000;
            border-radius: 5px;
        ">
            <h3>❌ Error retrieving weather for {city_name}</h3>
            <p>{data['error']}</p>
            <p>Please check the city name and your API key.</p>
        </div>
        """
        display(HTML(html_output))
        return

    try:
        location = data["location"]["name"]
        country = data["location"]["country"]
        temp_c = data["current"]["temp_c"]
        temp_f = data["current"]["temp_f"]
        condition_text = data["current"]["condition"]["text"]
        condition_icon = data["current"]["condition"]["icon"] # Get the weather icon URL
        last_updated = data["current"]["last_updated"]

        # Determine a background color based on temperature (simple example)
        if temp_c > 25:
            bg_color = "#ff9999" # Warm
        elif temp_c < 10:
            bg_color = "#99ccff" # Cool
        else:
            bg_color = "#99ff99" # Mild

        html_output = f"""
        <div style="
            background-color: {bg_color};
            border-radius: 15px;
            padding: 20px;
            margin: 20px auto;
            max-width: 400px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            color: #333;
            text-align: center;
        ">
            <h2 style="margin-top: 0; color: #333;">Weather in {location}, {country}</h2>
            <div style="display: flex; align-items: center; justify-content: center; margin-bottom: 10px;">
                <img src="http:{condition_icon}" alt="{condition_text}" style="width: 60px; height: 60px; margin-right: 10px;">
                <p style="font-size: 1.5em; margin: 0;">{condition_text}</p>
            </div>
            <p style="font-size: 2.5em; font-weight: bold; margin: 10px 0;">
                {temp_c}°C / {temp_f}°F
            </p>
            <p style="font-size: 0.9em; color: #555;">Last Updated: {last_updated}</p>
        </div>
        """
        display(HTML(html_output))

    except KeyError as e:
        html_output = f"""
        <div style="
            background-color: #ffe6e6;
            border-left: 6px solid #ff4d4d;
            padding: 10px;
            margin: 10px 0;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            color: #cc0000;
            border-radius: 5px;
        ">
            <h3>❌ Data Error for {city_name}</h3>
            <p>Missing expected data field: {e}</p>
            <p>API response structure might have changed or data is incomplete.</p>
        </div>
        """
        display(HTML(html_output))


# --- Interactive Input (for Jupyter) ---
from ipywidgets import interact, Text, Button, VBox, Output
from IPython.display import clear_output

# Create an input text field
city_input = Text(
    value='London',
    placeholder='Enter city name',
    description='City:',
    disabled=False
)

# Create a button
fetch_button = Button(description="Get Weather")

# Create an output widget to display results
output_area = Output()

def on_button_click(b):
    with output_area:
        clear_output(wait=True) # Clear previous output
        display_weather(city_input.value)

fetch_button.on_click(on_button_click)

# Display the widgets
display(VBox([city_input, fetch_button, output_area]))

In [2]:
import requests
import json
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk

# --- WeatherAPI Config ---
API_KEY = "876fd3d7689b415ea5310841231803"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

def get_weather_data(city):
    """Fetches weather data from the WeatherAPI."""
    params = {"key": API_KEY, "q": city}
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}
    except json.JSONDecodeError:
        return {"error": "Could not decode JSON from API response."}

def show_weather_window(city):
    data = get_weather_data(city)

    # Create new window
    win = tk.Toplevel(root)
    win.title(f"Weather in {city}")
    win.geometry("400x300")
    win.configure(bg="white")

    if "error" in data:
        tk.Label(win, text=f"❌ Error: {data['error']}",
                 fg="red", font=("Segoe UI", 12), bg="white").pack(pady=20)
        return

    try:
        location = data["location"]["name"]
        country = data["location"]["country"]
        temp_c = data["current"]["temp_c"]
        temp_f = data["current"]["temp_f"]
        condition_text = data["current"]["condition"]["text"]
        last_updated = data["current"]["last_updated"]

        # Background color logic
        if temp_c > 25:
            bg_color = "#ffcccc"
        elif temp_c < 10:
            bg_color = "#cce0ff"
        else:
            bg_color = "#ccffcc"

        win.configure(bg=bg_color)

        ttk.Label(win, text=f"Weather in {location}, {country}",
                  font=("Segoe UI", 14, "bold"), background=bg_color).pack(pady=10)
        ttk.Label(win, text=condition_text, font=("Segoe UI", 12), background=bg_color).pack(pady=5)
        ttk.Label(win, text=f"{temp_c}°C / {temp_f}°F", 
                  font=("Segoe UI", 16, "bold"), background=bg_color).pack(pady=10)
        ttk.Label(win, text=f"Last Updated: {last_updated}",
                  font=("Segoe UI", 10), background=bg_color).pack(pady=5)

    except KeyError as e:
        tk.Label(win, text=f"❌ Data Error: Missing field {e}",
                 fg="red", font=("Segoe UI", 12), bg="white").pack(pady=20)

# --- Tkinter Main Window ---
root = tk.Tk()
root.title("Weather App")
root.geometry("350x150")

city_var = tk.StringVar(value="London")

tk.Label(root, text="Enter City:", font=("Segoe UI", 12)).pack(pady=5)
city_entry = tk.Entry(root, textvariable=city_var, font=("Segoe UI", 12))
city_entry.pack(pady=5)

fetch_btn = tk.Button(root, text="Get Weather", font=("Segoe UI", 12),
                      command=lambda: show_weather_window(city_var.get()))
fetch_btn.pack(pady=10)

root.mainloop()
